In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import namedtuple

from vega_sim.null_service import VegaServiceNull


In [3]:
WalletConfig = namedtuple("WalletConfig", ["name", "passphrase"])

# Set up parties in the market/ Submit liquidity provision/ Control midprice
MM_WALLET = WalletConfig("mm", "pin")

# The party to send selling/buying MOs to hit LP orders
TRADER_WALLET = WalletConfig("Zl3pLs6Xk6SwIK7Jlp2x", "bJQDDVGAhKkj3PVCc7Rr")

# The party randomly post LOs at buy/sell side to simulate real Market situation
RANDOM_WALLET = WalletConfig("OJpVLvU5fgLJbhNPdESa", "GmJTt9Gk34BHDlovB7AJ")

# The party to terminate the market and send settlment price
TERMINATE_WALLET = WalletConfig("FJMKnwfZdd48C8NqvYrG", "bY3DxwtsCstMIIZdNpKs")

wallets = [MM_WALLET, TRADER_WALLET, RANDOM_WALLET, TERMINATE_WALLET]

In [4]:
vega = VegaServiceNull(run_wallet_with_console=True)
vega.start()

In [5]:
for wallet in wallets:
    vega.create_wallet(wallet.name, wallet.passphrase)

In [6]:
vega.mint(
    MM_WALLET.name,
    asset="VOTE",
    amount=10000000000,
)

vega.forward("10s")
vega.create_asset(
    MM_WALLET.name,
    name="tDAI",
    symbol="tDAI",
    decimals=5,
    max_faucet_amount=1e10,
)

In [7]:
tdai_id = vega.find_asset_id(symbol="tDAI")

In [8]:

vega.mint(
    MM_WALLET.name,
    tdai_id,
    amount=1e6
)
vega.mint(
    TERMINATE_WALLET.name,
    tdai_id,
    amount=1e2
)
vega.mint(
    RANDOM_WALLET.name,
    tdai_id,
    amount=1e6
)

In [9]:
vega.create_simple_market(
        market_name="BTC:DAI_Mar22",
        proposal_wallet=MM_WALLET.name,
        settlement_asset_id=tdai_id,
        termination_wallet=TERMINATE_WALLET.name,
    )

In [10]:
market_id = vega.all_markets()[0].id

In [11]:
market_id

'c4141503a6fbf8b64dc9cbfec1d40a91ce012908af8b8af206d89d50aad204c4'

In [21]:
%%timeit
vega.submit_order(
    trading_wallet=RANDOM_WALLET.name,
    market_id=market_id,
    time_in_force="TIME_IN_FORCE_GTC",
    order_type="TYPE_LIMIT",
    side="SIDE_SELL",
    volume=1,
    price=50000,
)

3.68 ms ± 402 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%%timeit
vega.submit_order(
    trading_wallet=RANDOM_WALLET.name,
    market_id=market_id,
    time_in_force="TIME_IN_FORCE_GTC",
    order_type="TYPE_LIMIT",
    side="SIDE_SELL",
    volume=1,
    price=50000,
    wait=False
)

2.91 ms ± 248 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
%%timeit
for _ in range(10):
    vega.market_info(market_id=market_id)

2.2 ms ± 92.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
vega.market_info(market_id=market_id)

[autoreload of vega_sim.service failed: Traceback (most recent call last):
  File "/Users/tom/.local/share/virtualenvs/vega-market-sim-Rx-j5lal/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/Users/tom/.local/share/virtualenvs/vega-market-sim-Rx-j5lal/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/Users/tom/.pyenv/versions/3.9.11/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 846, in exec_module
  File "<frozen importlib._bootstrap_external>", line 983, in get_code
  File "<frozen importlib._bootstrap_external>", line 913, in source_to_code
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/Users/tom/Code/vega-market-sim/vega_sim/

id: "c4141503a6fbf8b64dc9cbfec1d40a91ce012908af8b8af206d89d50aad204c4"
tradable_instrument {
  instrument {
    code: "BTC:DAI_Mar22"
    name: "BTC:DAI_Mar22"
    metadata {
      tags: "base:BTC"
    }
    future {
      settlement_asset: "42672ba8f605d9e51b1f9c399863c6e7feddb0a493f930ab3924422c68b5061a"
      quote_name: "BTC"
      oracle_spec_for_settlement_price {
        id: "d4bb865071002ab1e7a7d5160b7d6c419f67d5d304e26611f2b1f2e06e94c47d"
        pub_keys: "c5f874313d65a78bc2eba2ecf7780b024c5f74a9aedfe4be38cd39779896a6f2"
        filters {
          key {
            name: "price.BTC.value"
            type: TYPE_INTEGER
          }
        }
      }
      oracle_spec_for_trading_termination {
        id: "f24f5f9b18f6054b8834b71b4ae3d46e55eca2cdec443c1932e8b38bd31879da"
        pub_keys: "c5f874313d65a78bc2eba2ecf7780b024c5f74a9aedfe4be38cd39779896a6f2"
        filters {
          key {
            name: "trading.terminated"
            type: TYPE_BOOLEAN
          }
        